In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

loading the data


In [ ]:
train = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_train.csv', dtype=int)
X_train = train.drop('label', axis=1)
y_train = train['label']
test = pd.read_csv('/kaggle/input/mnist-in-csv/mnist_test.csv',dtype=int)
X_test = test.drop('label',axis=1)
y_test = test['label']

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
max_depth = [10,20,30,40,50,60]
depth_final =[]
oob_final =[]
def best_oob(CLF, X_train, y_train, n_estimators):
    for d in max_depth:
        c =CLF(n_estimators=n_estimators,max_depth=d,bootstrap=True,oob_score=True,n_jobs=-1,random_state=1502775)
        c.fit(X_train,y_train)
        oob_score = c.oob_score_
        oob_final.append(oob_score)
        depth_final.append(d)
        print('max depth = ',d,' oob score = ',oob_score)
    n = oob_final.index(max(oob_final))
    max_set = [oob_final[n],depth_final[n]]
    return max_set

In [ ]:
values = best_oob(ExtraTreesClassifier,X_train,y_train.values.ravel(),20)
print(values)

For the best oob score calculate the accuracy score

In [ ]:

from sklearn.metrics import accuracy_score
ETC = ExtraTreesClassifier(n_estimators=20,max_depth=50,bootstrap=True,oob_score=True,n_jobs=-1,random_state=1502775)
ETC.fit(X_train,y_train)
y_pred = ETC.predict(X_test)
score = accuracy_score(y_pred,y_test)
print("Accuracy score = ",score)


In [ ]:
oob_df = ETC.oob_decision_function_
pred_prob = ETC.predict_proba(X_test)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

def single_extra_tree(depth, seed=1502775):
    return ExtraTreesClassifier(max_depth=depth, n_estimators=1, bootstrap=False, random_state=seed)

In [ ]:
depth_final =[]
oob_final =[]
max_depth_boost = [10,20,30,40,50,60]
def best_oob_boost(X_train, y_train):
    for d in max_depth_boost:
        boost = AdaBoostClassifier(base_estimator=single_extra_tree(d), n_estimators=10, random_state=1502775)
        bag = BaggingClassifier(base_estimator=boost, n_estimators=30, bootstrap=True, oob_score=True, n_jobs=-1, random_state=1502775)
        bag.fit(X_train,y_train)
        oob_score = bag.oob_score_
        oob_final.append(oob_score)
        depth_final.append(d)
        print('max depth = ',d,' oob score = ',oob_score)
    n = oob_final.index(max(oob_final))
    max_set = [oob_final[n],depth_final[n]]
    return max_set




In [ ]:
values = best_oob_boost(X_train,y_train.values.ravel())
print(values)

calculating the accuracy score for the best depth and oobscore


In [ ]:
boost = AdaBoostClassifier(base_estimator=single_extra_tree(30), n_estimators=10, random_state=1502775)
bag = BaggingClassifier(base_estimator=boost, n_estimators=30, bootstrap=True, oob_score=True, n_jobs=-1, random_state=1502775)
bag.fit(X_train,y_train)
y_pred_bag=bag.predict(X_test)
score = accuracy_score(y_pred_bag,y_test)
print("Accuracy score = ",score)

In [ ]:
oob_df_bag = bag.oob_decision_function_
pred_prob_bag = bag.predict_proba(X_test)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
num_comp_final =[]
oob_final =[]
max_pca_comp = [20,40,60]
def best_oob_pca(X_train, y_train):
    for d in max_pca_comp:
        pca = PCA(n_components=d, svd_solver='randomized', random_state=1502775)
        pip = Pipeline( [ ('pca', pca),('ada_xt', AdaBoostClassifier(base_estimator=single_extra_tree(depth=30), n_estimators=10, random_state=1502775)) ])
        bag = BaggingClassifier(base_estimator=pip, n_estimators=30, bootstrap=True, oob_score=True, n_jobs=-1, random_state=1502775)
        bag.fit(X_train,y_train)
        oob_score = bag.oob_score_
        oob_final.append(oob_score)
        num_comp_final.append(d)
        print('num component = ',d,' oob score = ',oob_score)
    n = oob_final.index(max(oob_final))
    max_set = [oob_final[n],num_comp_final[n]]
    return max_set

In [ ]:
values = best_oob_pca(X_train,y_train.values.ravel())
print(values)

In [ ]:
pca = PCA(n_components=40, svd_solver='randomized', random_state=1502775)
pip = Pipeline( [ ('pca', pca),('ada_xt', AdaBoostClassifier(base_estimator=single_extra_tree(depth=30), n_estimators=10, random_state=1502775)) ])
bag = BaggingClassifier(base_estimator=pip, n_estimators=30, bootstrap=True, oob_score=True, n_jobs=-1, random_state=1502775)
bag.fit(X_train,y_train)
y_pred_pca = bag.predict(X_test)
score = accuracy_score(y_pred_pca,y_test)
print("Accuracy score = ",score)

In [ ]:
oob_df_pca = bag.oob_decision_function_
pred_prob_pca = bag.predict_proba(X_test)